In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import time 

In [0]:
seqs = pd.read_csv('/content/spike_protein_no_special.csv')
seqs.head(5)

,Sequences
0,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTG...
1,ATGTTGTTTTTCTTCTTTTTACACTTTGCCTTAGTAAATTCACAAT...
2,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
3,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...


In [0]:
seqs['Sequences']

0      ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTG...
1      ATGTTGTTTTTCTTCTTTTTACACTTTGCCTTAGTAAATTCACAAT...
2      ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
3      ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4      ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
                             ...                        
788    ATGGTGGACAGCCTTTGTTACTAATGTGAATGCGTCATCATCTGAA...
789    ATGGTTATGTCATGCATGCAAATTACATATTTTGGAGGAATACAAA...
790    ATGATATGATTTTATCTCTTCTTAGTAAAGGTAGACTTATAATTAG...
791    ATGGATGTTCCGTGGCTATAAAGATAACAGAACATTCTTGGAATGC...
792    ATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTT...
Name: Sequences, Length: 793, dtype: object

In [0]:
seq_list = seqs['Sequences'].tolist()
len(seq_list[0])

3819

In [0]:
#ignore 
combined = ''.join(seq_list)
print('Total characters: {}'.format(len(combined)))

Total characters: 3010860


In [0]:
vocab = sorted(set(combined))
print(vocab)
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for i, seq in enumerate(seq_list):
  seq_list[i] = np.array([char2idx[c] for c in seq])
print(seq_list)

['A', 'C', 'G', 'T']
[array([0, 3, 2, ..., 3, 3, 0]), array([0, 3, 2, ..., 3, 3, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 2, 0, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 3, 0, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..., 0, 1, 0]), array([0, 3, 2, ..

In [0]:
len(seq_list)

793

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100                                      #!!!
examples_per_epoch = len(combined)//(seq_length+1)    #!!!

# Create training examples / targets 
char_datasets = list(range(len(seq_list)))
for i, seq in enumerate(seq_list):
  char_datasets[i] = tf.data.Dataset.from_tensor_slices(seq_list[i])

for i in char_datasets[0].take(7):
  print(idx2char[i.numpy()])

A
T
G
T
T
T
G


In [0]:
sequences_list = list(range(len(char_datasets)))
for i, char_dataset in enumerate(char_datasets):
  sequences_list[i] = char_dataset.batch(seq_length+1, drop_remainder=True)  #!!!
# drop_remainder -> whether the last batch should be dropped in the case it has fewer than batch_size elements; the default behavior is not to drop the smaller batch
for item in sequences_list[0].take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTGTTAATCTAACAACTAGAACTCAGTTACCTCCTGCATACACCAACTCATCCACCCG'
'TGGTGTCTATTACCCTGACAAAGTTTTCAGATCTTCAGTTTTACATTTAACTCAGGATTTGTTTTTACCTTTCTTCTCCAATGTGACCTGGTTCCATGCTA'
'TACATGTTTCAGGGACCAATGGTATTAAAAGGTTTGATAACCCAGTTCTGCCATTCAACGATGGCGTCTATTTTGCTTCCACTGAGAAGTCTAATATAATA'
'AGAGGATGGATTTTTGGTACTACCTTAGATTCGAAGACCCAGTCTCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATT'
'TTGTAATGATCCATTTTTGGGTGTTTATTACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTACTCTAGTGCGAATAATTGCACTTTTG'


In [0]:
len(sequences_list)

793

In [0]:
#testing
a = sequences_list[0]
b = sequences_list[1]
c = sequences_list[2]
a

<BatchDataset shapes: (101,), types: tf.int64>

In [0]:
#testing; trying to concatenate batch datasets before mapping
ab = a.concatenate(b)
abc = ab.concatenate(c)
abc

<ConcatenateDataset shapes: (101,), types: tf.int64>

In [0]:
#testing; list(abc.as_numpy_iterator())  #uncomment and run to see output
len(list(abc.as_numpy_iterator())[0])

101

In [0]:
a = sequences_list[0]
for i in sequences_list[1:]:
  a = a.concatenate(i)
a 

<ConcatenateDataset shapes: (101,), types: tf.int64>

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = a.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTGTTAATCTAACAACTAGAACTCAGTTACCTCCTGCATACACCAACTCATCCACCC'
Target data: 'TGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTGTTAATCTAACAACTAGAACTCAGTTACCTCCTGCATACACCAACTCATCCACCCG'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('A')
  expected output: 3 ('T')
Step    1
  input: 3 ('T')
  expected output: 2 ('G')
Step    2
  input: 2 ('G')
  expected output: 3 ('T')
Step    3
  input: 3 ('T')
  expected output: 3 ('T')
Step    4
  input: 3 ('T')
  expected output: 3 ('T')


In [0]:
# Batch size
BATCH_SIZE = 64              #!!!

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000          #!!!

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 4) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           1024      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 4)             4100      
Total params: 3,943,428
Trainable params: 3,943,428
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([3, 3, 3, 1, 1, 1, 2, 2, 2, 1, 3, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 2,
       1, 1, 3, 2, 0, 1, 2, 3, 3, 2, 1, 2, 3, 1, 3, 1, 3, 0, 2, 2, 0, 2,
       3, 2, 1, 3, 2, 3, 0, 1, 2, 1, 1, 1, 1, 0, 3, 1, 3, 1, 3, 2, 3, 3,
       0, 1, 3, 2, 3, 1, 0, 3, 2, 1, 1, 0, 2, 2, 1, 0, 0, 1, 3, 0, 2, 0,
       0, 2, 0, 1, 1, 3, 3, 0, 0, 3, 2, 0])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'CATTTGTGGTGATTCAACTGAATGCAGCAATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTGCTTTAACTGGAATAGCTGTTGAACAA'

Next Char Predictions: 
 'TTTCCCGGGCTCCCCTCTCTCGCCTGACGTTGCGTCTCTAGGAGTGCTGTACGCCCCATCTCTGTTACTGTCATGCCAGGCAACTAGAAGACCTTAATGA'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 4)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       1.3845732


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
455/455 [==============================] - 21s 47ms/step - loss: 0.5050
Epoch 2/10
455/455 [==============================] - 21s 46ms/step - loss: 0.0483
Epoch 3/10
455/455 [==============================] - 21s 47ms/step - loss: 0.0429
Epoch 4/10
455/455 [==============================] - 21s 47ms/step - loss: 0.0412
Epoch 5/10
455/455 [==============================] - 21s 46ms/step - loss: 0.0396
Epoch 6/10
455/455 [==============================] - 21s 47ms/step - loss: 0.0387
Epoch 7/10
455/455 [==============================] - 21s 46ms/step - loss: 0.0379
Epoch 8/10
455/455 [==============================] - 21s 47ms/step - loss: 0.0376
Epoch 9/10
455/455 [==============================] - 21s 46ms/step - loss: 0.0373
Epoch 10/10
455/455 [==============================] - 21s 46ms/step - loss: 0.0369


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            1024      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 4)              4100      
Total params: 3,943,428
Trainable params: 3,943,428
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3771                                       #!!!

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0                                           #!!!

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTGTT"))

ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTTTCTAGTCAGTGTGTTAATCTTACAACCAGAATTCAGTTACCTCCTGCATACACTAATTCTTTCACACGGAGTTACACAGAATGTTCTCTATGAGAACCAAAAATTGATTGCCAACCAATTTAATAGTGCTATTGGCAAAATTCAAGACTCACTTTCTTCCACAGCAAGGAGTTACACAGAATGTTCTCTATGAGAACCAAAAATTGATTGCCAACCAATTTAATAGTGCTATTGGCAAAATTCAAGACTCACTTTCTTCCACAGCAAGATGAAGTCAGACAAATCGCTCCAGGGCAAACTGGAAAGATTGCTGATTATAATTATAAATTACCAGATGATTTTACAGGCTGCGTTATAGCTTGGAATTCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGGTTTCTAATAACTCTATTGCCATACCCACAAATTTTACAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAGACTCATTCAAGGAGGAGTTAGATAAATATTTTAAGAATCATACATCACCAGATGTTGATTTAGGTGACATCTCTGGCATTAATGCTTCAGTTGTAAACATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGTGTTTTTTTGGTCCGCATCATTTTCCACTTTTAAGTGTTATGGAGTGTCTCCTACTAAATTAAATGATCTCTGCTTTACTAATGTCTATGCAGATTCATTTGTAATTAGAGGTAATGTTTTTCAAACACGTGCAGGCTGTTTAATAGGGGCTGAACATGTCAACAACTCATATGAGTGTGACATACCCATTGGTGCAGGTATATGCGCTAGTTCTAT